# Phase 1 Project API Calls #

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

%matplotlib inline

In [2]:
#import imdb title basics into df
file='zippedData/imdb.title.basics.csv.gz'
imdb_df=pd.read_csv(file)
imdb_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [3]:
imdb_df['start_year'].value_counts()

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2027        1
2026        1
2025        1
2115        1
Name: start_year, dtype: int64

In [10]:
imdb_10yr=imdb_df[imdb_df['start_year']<2021]
imdb_10yr.shape

(146018, 6)

In [11]:
import json

In [13]:
#load key for IMDB API
def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/morgandj/.secret/imdb_creds.json")
host=keys['host']
key=keys['key']

In [25]:
#function that calls data on a list of movie ids from a specific endpoint in imdb
#returns a list of responses
def imdb_requests_byid(list_ids,url):
    responses=[]
    for count,i in enumerate(list_ids):
        imdb_id=i
        #if (count%1000)==0:
         #   print(count)
        params={'tconst': imdb_id}
        response=requests.get(url,headers=headers, params=params)
        responses.append(response)
    return responses

In [15]:
#make list of all ids for movies from last 10 years
imdb_10_ids=imdb_10yr['tconst']
id_list=imdb_10_ids.tolist()
len(id_list)

146018

In [24]:
#break the longer list into 500 id chunks 
id_list_of_lists=[id_list[(i-500):i] for i in list(range(500,147000,500))]
len(id_list_of_lists[-1])
len(id_list_of_lists)

293

In [26]:
#query versions url to get country data
get_versions_url='https://imdb8.p.rapidapi.com/title/get-versions'
headers={'x-rapidapi-host': host, 
         'x-rapidapi-key': key}
all_version_responses=[]

for count, id_list_ in enumerate(id_list_of_lists):
    print(count)
    version_responses=imdb_requests_byid(id_list_,get_versions_url)
    all_version_responses.extend(version_responses)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
all_version_json=[x.json() for x in all_version_responses if x.status_code==200]

In [33]:
len(all_version_json)

145917

In [37]:
jsonString=json.dumps(all_version_json)
jsonFile=open('imdb_versions.json','w')
jsonFile.write(jsonString)
jsonFile.close()

In [38]:
all_version_json[:5]

[{'@type': 'imdb.api.title.versions',
  'alternateTitles': [{'language': 'en',
    'region': 'IN',
    'title': 'Sunghursh',
    'types': ['imdbDisplay']},
   {'attributes': ['alternative transliteration'],
    'language': 'hi',
    'region': 'IN',
    'title': 'Sangharsh'},
   {'title': 'Sunghursh', 'types': ['original']},
   {'attributes': ['alternative transliteration'],
    'language': 'hi',
    'region': 'IN',
    'title': 'Sunghursh'},
   {'attributes': ['alternative spelling'],
    'language': 'hi',
    'region': 'IN',
    'title': 'Sungharsh'}],
  'colorations': ['color'],
  'defaultTitle': 'Sunghursh',
  'silent': False,
  'spokenLanguages': ['hi'],
  'originalTitle': 'Sunghursh',
  'origins': ['IN'],
  'runningTimes': [{'timeMinutes': 175}],
  'id': '/title/tt0063540/',
  'title': 'Sunghursh',
  'titleType': 'movie',
  'year': 1968},
 {'@type': 'imdb.api.title.versions',
  'alternateTitles': [{'language': 'en',
    'region': 'IN',
    'title': 'Ashad Ka Ek Din',
    'types': 

In [39]:
pd.DataFrame.from_records(all_version_json)

AttributeError: 'NoneType' object has no attribute 'keys'

In [40]:
#create a list of dictionaries with keys tconst and country
#convert into df

country_dictionaries=[]

for v in all_version_json:
    if (v != None) and ('origins' in v.keys()) and ('id' in v.keys()):
        country_dict={}
        tconst=v['id']
        if tconst != None:
            country_dict['tconst']=tconst[7:16]
        else:
            country_dict['tconst']=None
        country=v['origins']
        if country != None:
            country_dict['country']=country[0]
        else:
            country_dict['country']=None
        country_dictionaries.append(country_dict)
    
country_df=pd.DataFrame(country_dictionaries)
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142928 entries, 0 to 142927
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   tconst   142928 non-null  object
 1   country  142928 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [41]:
country_df.to_csv('imdb_countries.csv')

In [44]:
us_movies=country_df[country_df['country']=='US']

In [65]:
us_movies.shape

(49608, 2)

In [46]:
us_id_list=us_movies['tconst'].tolist()
us_id_list[:10]

['tt0112502',
 'tt0159369',
 'tt0192528',
 'tt0230212',
 'tt0249516',
 'tt0255820',
 'tt0285252',
 'tt0293069',
 'tt0297400',
 'tt0306058']

In [50]:
#break the longer list into 500 id chunks to help me see where the errors are
#wasn't able to run query on everything at once
us_id_lists=[us_id_list[(i-500):i] for i in list(range(500,50500,500))]
len(us_id_lists[-1])

108

In [51]:
len(us_id_lists)

100

In [52]:
#query versions url to get country data
imdb_business_url='https://imdb8.p.rapidapi.com/title/get-business'
headers={'x-rapidapi-host': host, 
         'x-rapidapi-key': key}
all_business_responses=[]

for count, id_list_ in enumerate(us_id_lists):
    print(count)
    business_responses=imdb_requests_byid(id_list_,imdb_business_url)
    all_business_responses.extend(business_responses)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [54]:
all_business_responses[5].content

b'{"@meta":{"operation":"TitleBusinessV2","requestId":"e5a4940a-5b32-46dc-b776-5fd0ecd3818f","serviceTimeMs":23.360727},"resource":{"@type":"imdb.api.title.v2.business","id":"/title/tt0255820/","title":"Return to Babylon","titleType":"movie","year":2013,"budget":{"amount":2000000.0,"currency":"USD"}}}'

In [55]:
#write function that takes a business reponse and returns dictionary of values
def prepare_bus_response(response):
    resp_json=response.json()
    if 'resource' in resp_json.keys():
        budg_dict={}
        resp_json=resp_json['resource']
        if 'id' in resp_json.keys():
            budg_dict['tconst']=resp_json['id'][7:16]
        if 'budget' in resp_json.keys():
            budg_dict['budget']=resp_json['budget']
        if 'gross' in resp_json.keys():
            if 'aggregations' in resp_json['gross'].keys():
                gross_agg=resp_json['gross']['aggregations']
                for area in gross_agg:
                    if area['area']=='XWW':
                        budg_dict['ww_gross']=area['total']['amount']
                        budg_dict['ww_currency']=area['total']['currency']
                    if area['area']=='XNDOM':
                        budg_dict['int_gross']=area['total']['amount']
                        budg_dict['int_currency']=area['total']['currency']
            if 'regional' in resp_json['gross']:
                gross_region=resp_json['gross']['regional']
                for region in gross_region:
                    if region['region']=='XDOM':
                        budg_dict['domestic_gross']=region['total']['amount']
                        budg_dict['domestic_currency']=region['total']['currency']
        return budg_dict

In [56]:
business_dicts=[]
for r in all_business_responses:
    if r.status_code==200:
        dictionary=prepare_bus_response(r)
        business_dicts.append(dictionary)


In [61]:
len(business_dicts)

49564

In [59]:
business_dicts_notNa=[x for x in business_dicts if x != None]
len(business_dicts_notNa)

18967

In [60]:
imdb_business=pd.DataFrame(business_dicts_notNa)

In [62]:
imdb_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18967 entries, 0 to 18966
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             18967 non-null  object 
 1   budget             16717 non-null  object 
 2   int_gross          2742 non-null   float64
 3   int_currency       2742 non-null   object 
 4   ww_gross           4205 non-null   float64
 5   ww_currency        4205 non-null   object 
 6   domestic_gross     3473 non-null   float64
 7   domestic_currency  3473 non-null   object 
dtypes: float64(3), object(5)
memory usage: 1.2+ MB


In [63]:
imdb_business.to_csv('imdb_business.csv')

In [64]:
#inner join all the US titles with imdb 10_yr to get all movies made in US in last 10yrs
imdb=imdb_10yr.merge(us_movies, on='tconst')
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48305 entries, 0 to 48304
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           48305 non-null  object 
 1   primary_title    48305 non-null  object 
 2   original_title   48305 non-null  object 
 3   start_year       48305 non-null  int64  
 4   runtime_minutes  39168 non-null  float64
 5   genres           47628 non-null  object 
 6   country          48305 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.9+ MB


In [72]:
#drop duplicates
imdb=imdb.drop_duplicates(subset='tconst')
imdb.shape

(48203, 7)

In [73]:
imdb=imdb.merge(imdb_business,on='tconst',how='left')
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48246 entries, 0 to 48245
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             48246 non-null  object 
 1   primary_title      48246 non-null  object 
 2   original_title     48246 non-null  object 
 3   start_year         48246 non-null  int64  
 4   runtime_minutes    39126 non-null  float64
 5   genres             47570 non-null  object 
 6   country            48246 non-null  object 
 7   budget             16661 non-null  object 
 8   int_gross          2728 non-null   float64
 9   int_currency       2728 non-null   object 
 10  ww_gross           4188 non-null   float64
 11  ww_currency        4188 non-null   object 
 12  domestic_gross     3467 non-null   float64
 13  domestic_currency  3467 non-null   object 
dtypes: float64(4), int64(1), object(9)
memory usage: 5.5+ MB


In [76]:
imdb=imdb.drop_duplicates(subset='tconst')
imdb.shape

(48203, 14)

### Merge data with provided boxoffice gross datasets ###

In [77]:
bom_df=pd.read_csv('zippedData/bom.movie_gross.csv.gz')
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [79]:
bom_df.rename(columns={'domestic_gross':'dom_gross_bom'},inplace=True)
bom_df.columns

Index(['title', 'studio', 'dom_gross_bom', 'foreign_gross', 'year'], dtype='object')

In [80]:
#merge with imdb on original title
imdb=imdb.merge(bom_df,left_on='original_title',right_on='title',how='left')
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48203 entries, 0 to 48202
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             48203 non-null  object 
 1   primary_title      48203 non-null  object 
 2   original_title     48203 non-null  object 
 3   start_year         48203 non-null  int64  
 4   runtime_minutes    39092 non-null  float64
 5   genres             47527 non-null  object 
 6   country            48203 non-null  object 
 7   budget             16622 non-null  object 
 8   int_gross          2723 non-null   float64
 9   int_currency       2723 non-null   object 
 10  ww_gross           4182 non-null   float64
 11  ww_currency        4182 non-null   object 
 12  domestic_gross     3465 non-null   float64
 13  domestic_currency  3465 non-null   object 
 14  title              1526 non-null   object 
 15  studio             1526 non-null   object 
 16  dom_gross_bom      151

In [82]:
imdb['ww_currency'].value_counts()

USD    4181
INR       1
Name: ww_currency, dtype: int64

In [83]:
imdb[imdb['ww_currency']=='INR']

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,country,budget,int_gross,int_currency,ww_gross,ww_currency,domestic_gross,domestic_currency,title,studio,dom_gross_bom,foreign_gross,year
32275,tt5119108,Highway 5,Highway 5,2016,116.0,"Comedy,Drama",US,NaN,NaN,NaN,150000000.0,INR,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
#drop movie not in USD currency
imdb=imdb[imdb['ww_currency']!='INR']

In [85]:
imdb['domestic_currency'].value_counts()

USD    3465
Name: domestic_currency, dtype: int64

In [88]:
#drop currency columns
imdb=imdb.drop(['ww_currency','domestic_currency'],axis=1)
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48202 entries, 0 to 48202
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           48202 non-null  object 
 1   primary_title    48202 non-null  object 
 2   original_title   48202 non-null  object 
 3   start_year       48202 non-null  int64  
 4   runtime_minutes  39091 non-null  float64
 5   genres           47526 non-null  object 
 6   country          48202 non-null  object 
 7   budget           16622 non-null  object 
 8   int_gross        2723 non-null   float64
 9   int_currency     2723 non-null   object 
 10  ww_gross         4181 non-null   float64
 11  domestic_gross   3465 non-null   float64
 12  title            1526 non-null   object 
 13  studio           1526 non-null   object 
 14  dom_gross_bom    1519 non-null   float64
 15  foreign_gross    1112 non-null   object 
 16  year             1526 non-null   float64
dtypes: float64(6

In [90]:
imdb=imdb.drop(['int_currency'],axis=1)

In [91]:
imdb['domestic_gross']=imdb['domestic_gross'].fillna(imdb['dom_gross_bom'])
imdb['int_gross']=imdb['int_gross'].fillna(imdb['foreign_gross'])
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48202 entries, 0 to 48202
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           48202 non-null  object 
 1   primary_title    48202 non-null  object 
 2   original_title   48202 non-null  object 
 3   start_year       48202 non-null  int64  
 4   runtime_minutes  39091 non-null  float64
 5   genres           47526 non-null  object 
 6   country          48202 non-null  object 
 7   budget           16622 non-null  object 
 8   int_gross        2888 non-null   object 
 9   ww_gross         4181 non-null   float64
 10  domestic_gross   3728 non-null   float64
 11  title            1526 non-null   object 
 12  studio           1526 non-null   object 
 13  dom_gross_bom    1519 non-null   float64
 14  foreign_gross    1112 non-null   object 
 15  year             1526 non-null   float64
dtypes: float64(5), int64(1), object(10)
memory usage: 6.3+ MB


In [94]:
#fix int_gross to float in order to fill ww_gross with sum of int and domestic
def clean_string(object):
    if type(object)==str:
        return float(object.replace('$','').replace(',',''))
    else: 
        return object

imdb['int_gross']=imdb['int_gross'].apply(clean_string)


,tconst,primary_title,original_title,start_year,runtime_minutes,genres,country,budget,int_gross,ww_gross,domestic_gross,title,studio,dom_gross_bom,foreign_gross,year
0,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller",US,"{'amount': 1300000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0159369,Cooper and Hemingway: The True Gen,Cooper and Hemingway: The True Gen,2013,180.0,Documentary,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0192528,Heaven & Hell,Reverse Heaven,2018,104.0,Drama,US,"{'amount': 5000000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0249516,Foodfight!,Foodfight!,2012,91.0,"Action,Animation,Comedy",US,"{'amount': 65000000.0, 'currency': 'USD'}",120141.0,120141.0,NaN,NaN,NaN,NaN,NaN,NaN
5,tt0255820,Return to Babylon,Return to Babylon,2013,75.0,"Biography,Comedy,Drama",US,"{'amount': 2000000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,tt0285252,Life's a Beach,Life's a Beach,2012,100.0,Comedy,US,"{'amount': 4000000.0, 'currency': 'USD'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,tt0293069,Dark Blood,Dark Blood,2012,86.0,Thriller,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,tt0297400,Snowblind,Snowblind,2015,NaN,"Crime,Drama",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tt0306058,Second Coming,Second Coming,2012,95.0,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#fill ww_gross missing values where int and domestic exist
imdb['ww_gross']=np.where(imdb['ww_gross'].isnull() & 
                                  imdb['domestic_gross'].notnull() & 
                                  imdb['int_gross'].notnull(),
                                  imdb['domestic_gross']+imdb['int_gross'],
                                  imdb['ww_gross'])
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48202 entries, 0 to 48202
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           48202 non-null  object 
 1   primary_title    48202 non-null  object 
 2   original_title   48202 non-null  object 
 3   start_year       48202 non-null  int64  
 4   runtime_minutes  39091 non-null  float64
 5   genres           47526 non-null  object 
 6   country          48202 non-null  object 
 7   budget           16622 non-null  object 
 8   int_gross        2888 non-null   float64
 9   ww_gross         4343 non-null   float64
 10  domestic_gross   3728 non-null   float64
 11  title            1526 non-null   object 
 12  studio           1526 non-null   object 
 13  dom_gross_bom    1519 non-null   float64
 14  foreign_gross    1112 non-null   object 
 15  year             1526 non-null   float64
dtypes: float64(6), int64(1), object(9)
memory usage: 6.3+ MB


In [98]:
imdb=imdb[imdb['ww_gross'].notnull()]

In [99]:
imdb=imdb.drop(['primary_title','country','title','studio','dom_gross_bom',
               'foreign_gross','year','int_gross','domestic_gross'],axis=1)

In [102]:
#drop runtimes under 45 minutes and over 4 hours

imdb=imdb[(imdb['runtime_minutes']>45.0) & (imdb['runtime_minutes']<240.0)]
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4065 entries, 4 to 48185
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           4065 non-null   object 
 1   original_title   4065 non-null   object 
 2   start_year       4065 non-null   int64  
 3   runtime_minutes  4065 non-null   float64
 4   genres           4059 non-null   object 
 5   budget           1957 non-null   object 
 6   ww_gross         4065 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 254.1+ KB


Now we have all the worldwide gross data that we could get from provided data and the IMDB API. I dropped all movies that did not have this data since it is essential to the analysis.  Now we will collect any more budget date can find as well as release date and MPAA rating.  I dropped the other rows first to limit the number of API calls.

In [103]:
imdb_ids=imdb['tconst'].to_list()
len(imdb_ids)

4065

In [104]:
imdb.to_csv('27SEP_imdb.csv')

In [ ]:
#get tmdb id for these movies
